# モデルの精度を確認するためのインターフェース

ver.2022/03/19

## 1 概要
「Train_RT-MMVC.ipynb」で学習したモデルでTTSと非リアルタイムのVCを行い、モデルの精度を検証します。

## 2 Google Drive をマウント
**Google Drive にアップロードした RT-MMVC_Trainer を参照できるように、設定します。**

「このノートブックに Google ドライブのファイルへのアクセスを許可しますか？」

といったポップアップが表示されるので、「Google ドライブに接続」を押下し、google アカウントを選択して、「許可」を選択してください。

成功すれば、下記メッセージが出ます。
```
Mounted at /content/drive/
```


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

cdコマンドを実行して、マウントしたGoogle Drive のRT-MMVC_Trainerディレクトリに移動します。

%cd 「RT-MMVC_Trainerをgoogle driveにパップロードしたパス」

としてください。

正しいパスが指定されていれば

-rw------- 1 root root 11780 Mar  4 16:53 attentions.py

-rw------- 1 root root  4778 Mar  4 16:53 commons.py

drwx------ 2 root root  4096 Mar  5 15:20 configs

...といった感じに表示されるはずです。

In [ ]:
%cd /content/drive/MyDrive/Colab Notebooks/
!ls -la

## 3 必要なライブラリのインストール

何も考えず実行してください。

In [ ]:
!apt-get install espeak
!pip install Cython==0.29.21
!pip install librosa==0.8.0
!pip install matplotlib==3.3.1
!pip install numpy==1.18.5
!pip install phonemizer==2.2.1
!pip install scipy==1.5.2
!pip install tensorboard==2.3.0
!pip install torch==1.6.0
!pip install torchvision==0.7.0
!pip install Unidecode==1.1.1
!pip install retry

In [ ]:
!pip install pyopenjtalk

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import IPython.display as ipd

import os
import json
import math
import torch
from torch import nn
from torch.nn import functional as F
from torch.utils.data import DataLoader

import commons
import utils
from data_utils import TextAudioLoader, TextAudioCollate, TextAudioSpeakerLoader, TextAudioSpeakerCollate
from models import SynthesizerTrn
from text.symbols import symbols
from text import text_to_sequence

from scipy.io.wavfile import write


def get_text(text, hps):
    text_norm = text_to_sequence(text, hps.data.text_cleaners)
    if hps.data.add_blank:
        text_norm = commons.intersperse(text_norm, 0)
    text_norm = torch.LongTensor(text_norm)
    return text_norm

In [ ]:
import pyopenjtalk
def mozi2phone(mozi):
    text = pyopenjtalk.g2p(mozi)
    text = "sil " + text + " sil"
    text = text.replace(' ', '-')
    return text

## 4 学習したモデルを読み込む

CONFIG_PATH に学習に利用したjsonファイルを`「./configs/****.json」`のように指定し、

NET_PATHに学習したモデルを`「./configs/xxxx/G_*****.pth」`のように指定してください。

In [ ]:
CONFIG_PATH = "./configs/baseconfig_seg4192.json"
NET_PATH = "./logs/20220306_24000/G_348000.pth"

指定したファイルをもとにモデルの読み込みを行います。

In [ ]:
hps = utils.get_hparams_from_file(CONFIG_PATH)

In [ ]:
net_g = SynthesizerTrn(
    len(symbols),
    hps.data.filter_length // 2 + 1,
    hps.train.segment_size // hps.data.hop_length,
    n_speakers=hps.data.n_speakers,
    **hps.model)
_ = net_g.eval()

_ = utils.load_checkpoint(NET_PATH, net_g, None)

## 5 学習したモデルでTTSを行う

TEXTに**ひらがな**で文章を入力してください。

In [ ]:
TEXT = "おはよう、しょくん。わたしはてんのうずあいるだ。これはおんせいごうせいのてすとだ。どうだ？すごいだろう。"

TTS_Speaker_IDにTTSを実行する話者IDを代入してください。  
(filelists/xxxx_Correspondence.txtに話者Idと対応した話者の対応が出力されています！)

In [ ]:
TTS_Speaker_ID = 0

実際にTTSを行います。

イントネーションがおかしくなることが多々ありますが、VC的には問題なので気にしなくて大丈夫です。

初回のみpyopenjtalk周りのダウンロードが始まるので、少し時間がかかりますが仕様です。

In [ ]:
text = TEXT
stn_tst = get_text(mozi2phone(text), hps)

with torch.no_grad():
    x_tst = stn_tst.unsqueeze(0)
    x_tst_lengths = torch.LongTensor([stn_tst.size(0)])
    sid = torch.LongTensor([TTS_Speaker_ID])
    audio = net_g.infer(x_tst, x_tst_lengths, sid=sid, noise_scale=.400, noise_scale_w=0.8, length_scale=1)[0][0,0].data.cpu().float().numpy()
ipd.display(ipd.Audio(audio, rate=hps.data.sampling_rate))

## 6 学習したモデルで非リアルタイムVCを行う

非リアルタイムのVCを行います。

ソース話者のIDとその話者の音声ファイルのパス、変換ターゲットの話者のIDを指定してください。

In [ ]:
SOURCE_WAVFILE = "dataset/textful/000_jvs001/wav/VOICEACTRESS100_001.wav"
SOURCE_SPEAKER_ID = 0
TARGET_ID = 1

実際にVCを行います。

ここでの性能が悪い場合、学習不足か他に問題があります。

In [ ]:
with torch.no_grad():
    dataset = TextAudioSpeakerLoader(hps.data.validation_files_notext, hps.data)
    data = dataset.get_audio_text_speaker_pair([SOURCE_WAVFILE, SOURCE_SPEAKER_ID, "a"])
    data = TextAudioSpeakerCollate()([data])
    x, x_lengths, spec, spec_lengths, y, y_lengths, sid_src = [x for x in data]
    sid_tgt1 = torch.LongTensor([TARGET_ID])
    audio1 = net_g.voice_conversion(spec, spec_lengths, sid_src=sid_src, sid_tgt=sid_tgt1)[0][0,0].data.cpu().float().numpy()
print("Original SID: %d" % sid_src.item())
ipd.display(ipd.Audio(y[0].cpu().numpy(), rate=hps.data.sampling_rate))
print("Converted SID: %d" % sid_tgt1.item())
ipd.display(ipd.Audio(audio1, rate=hps.data.sampling_rate))